## 1. Introduction

This work presents a comparison between classical machine learning methods and a neural network built from scratch for binary classification. The goal is to understand how both approaches perform on a real-world physics problem and to identify situations where one method may work better than the other.

Classical machine learning algorithms like logistic regression and support vector machines have been used successfully for many years (Hastie et al., 2009). These methods work well when data has clear patterns and when the number of features is not too large. Neural networks, on the other hand, can learn more complex patterns but need more data and careful training (Goodfellow et al., 2016). By building a neural network from scratch using only NumPy, this analysis aims to show exactly how these models learn and where their strengths come from.

The analysis follows several steps. First, a dataset is selected and explored to understand its properties. Second, two classical ML models are trained with different settings to find what works best. Third, a 3-layer feed-forward neural network is built from scratch, with all calculations done manually. Finally, all models are compared using standard metrics like accuracy, precision, recall, and F1-score. This approach allows for a clear understanding of both the theory and practice of these different methods.

## 2. Dataset Selection and Justification

### 2.1 Choice of Dataset

The MAGIC Gamma Telescope dataset from the UCI Machine Learning Repository serves as the foundation for this analysis (Bock, 2007). This dataset comes from the MAGIC telescope system, which detects high-energy gamma rays from cosmic sources like supernova remnants and active galactic nuclei.

### 2.2 Scientific Background

When gamma rays from space enter Earth's atmosphere, they create showers of particles that produce brief flashes of light. Ground-based telescopes capture these flashes to study distant cosmic events. However, the telescope also detects similar flashes from hadrons, which are different types of particles. The challenge is to separate true gamma ray signals from hadron backgrounds, which is critical for astronomical observations (Albert et al., 2008).

This problem is formulated as a binary classification task. Each observation must be labeled as either a gamma ray event (signal class) or a hadron event (background class). Success in this task directly impacts the ability of astronomers to detect and study gamma ray sources in the universe.

### 2.3 Dataset Structure

The dataset contains 19,020 observations with 10 numerical features. Each feature describes the shape and properties of the light flash captured by the telescope. The features are:

1. fLength: length of the light pattern in millimeters
2. fWidth: width of the light pattern in millimeters
3. fSize: total brightness measured as photon count
4. fConc: concentration ratio of the brightest pixels
5. fConc1: concentration of the single brightest pixel
6. fAsym: measure of how lopsided the pattern is
7. fM3Long: third moment of the pattern along its length
8. fM3Trans: third moment of the pattern along its width
9. fAlpha: angle of the pattern relative to the source
10. fDist: distance from the center in millimeters

The dataset contains 12,332 gamma ray events and 6,688 hadron events, giving a class distribution of approximately 65% gamma and 35% hadron.

### 2.4 Justification for Dataset Selection

This dataset was selected for several reasons. First, it addresses a genuine scientific problem in astrophysics rather than being an artificial test case. The ability to classify these events accurately has real implications for gamma ray astronomy. Second, the features are based on physical measurements and have clear interpretations, which helps in understanding model behavior. Third, the class imbalance reflects realistic conditions that occur in actual telescope operations. Most classification benchmarks use balanced classes, but real-world problems often have unequal class sizes, making this dataset more representative of practical challenges.

Finally, this dataset is not among the most commonly used benchmarks like iris or Titanic, which have been analyzed extensively. Using a less common dataset ensures that the analysis provides fresh insights rather than repeating well-known results.

**Citation:**  
Bock, R. K. (2007). MAGIC Gamma Telescope Dataset. UCI Machine Learning Repository. https://doi.org/10.24432/C52C8B

## 3. Data Loading and Exploratory Analysis

Before building any models, it is necessary to load the dataset and examine its basic properties. This section explores the data structure, checks for missing values, and looks at how features are distributed across the two classes. Understanding these patterns helps guide decisions about preprocessing and model selection.

The code below loads the dataset directly from the UCI repository and assigns meaningful column names to each feature. After loading, basic statistics are computed to check for any unusual values or data quality issues.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for consistent visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load the MAGIC Gamma Telescope dataset
# Dataset available at: https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data"

column_names = [
    'fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 
    'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class'
]

df = pd.read_csv(url, names=column_names)

print("Dataset loaded successfully.")
print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
df.head()

In [ ]:
# Basic statistical summary
print("Dataset Information:")
print(df.info())
print("\nStatistical Summary:")
df.describe()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Examine class distribution
print("\nClass distribution:")
print(df['class'].value_counts())
print("\nClass proportions:")
print(df['class'].value_counts(normalize=True))

### 3.1 Data Visualization

Visual analysis of the feature distributions provides insight into which features might be most useful for classification. The plots below show how each feature is distributed for gamma ray events compared to hadron events. Features that show clear separation between the two classes are likely to be strong predictors in the classification models.

In [ ]:
# Visualize class distribution
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
class_counts = df['class'].value_counts()
ax.bar(class_counts.index, class_counts.values, color=['#2ecc71', '#e74c3c'])
ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Class Distribution: Gamma vs Hadron Events', fontsize=14)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Feature distribution analysis
fig, axes = plt.subplots(2, 5, figsize=(18, 8))
axes = axes.ravel()

for idx, col in enumerate(df.columns[:-1]):
    axes[idx].hist(df[df['class'] == 'g'][col], bins=30, alpha=0.6, label='Gamma', color='blue')
    axes[idx].hist(df[df['class'] == 'h'][col], bins=30, alpha=0.6, label='Hadron', color='red')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].legend(fontsize=8)
    axes[idx].grid(alpha=0.3)

plt.suptitle('Feature Distributions by Class', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 3.2 Interpretation of Feature Distributions


The histograms reveal several important patterns in the data. Features such as fLength, fWidth, fSize, and fConc show different distributions for gamma rays compared to hadrons. This suggests that these features carry information that can help separate the two classes. However, the distributions still show considerable overlap, which means that no single feature can perfectly separate gamma rays from hadrons on its own.Given these patterns, it is expected that classification models will benefit from using all features together rather than relying on any single measurement. The overlap in distributions also suggests that perfect classification is unlikely, and some level of error should be expected even with good models.


The fAlpha feature shows particularly strong separation between the two classes. This observation aligns with findings in the astrophysics literature, where the alpha parameter has been identified as one of the most powerful features for gamma ray identification (Albert et al., 2008). The physical reason for this is that gamma ray showers tend to point back toward their source, creating a characteristic angular signature that differs from hadron showers.

## References

Albert, J., Aliu, E., Anderhub, H., Antoranz, P., Armada, A., Asensio, M., ... & Baixeras, C. (2008). Implementation of the random forest method for the imaging atmospheric Cherenkov telescope MAGIC. Nuclear Instruments and Methods in Physics Research Section A, 588(3), 424-432.

Bock, R. K. (2007). MAGIC Gamma Telescope Dataset. UCI Machine Learning Repository. https://doi.org/10.24432/C52C8B

Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. MIT Press. http://www.deeplearningbook.org

Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., ... & Duchesnay, E. (2011). Scikit-learn: Machine learning in Python. Journal of Machine Learning Research, 12, 2825-2830.

Hastie, T., Tibshirani, R., & Friedman, J. (2009). The Elements of Statistical Learning: Data Mining, Inference, and Prediction (2nd ed.). Springer.